CRAWL DATA

In [1]:
import json
import os
import re
import pandas as pd
from Libraries.Sorter import ArticleSorter
from Libraries.Crawler import CategoryValidator, UrlCollector, ArticleCrawler

In [2]:
# === CÁC HÀM XỬ LÝ FILE ===

def load_json(file_path):
    if not os.path.exists(file_path):
        return []
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)
    
def replace_json(data, file_path):
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

def save_json(data, file_path):
    with open(file_path, 'a', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

def load_jsonl(file_path):
    data = []
    if os.path.exists(file_path):
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                data.append(json.loads(line))
    return data

def replace_jsonl(data, file_path):
    with open(file_path, 'w', encoding='utf-8') as f:
        for item in data:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')

def save_jsonl(data, file_path):
    with open(file_path, 'a', encoding='utf-8') as f:
        for item in data:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')


# === CÁC HÀM HỖ TRỢ ===

def get_urls_from_url_file(file_path):
    """Lấy set các URL đã có từ file URLS.json."""
    urls = set()
    # Hàm load_json trả về một list các dictionary
    url_info_list = load_json(file_path) 
    for item in url_info_list:
        if 'url' in item:
            urls.add(item['url'])
    return urls

def get_existing_article_urls(file_path):
    """Lấy set các URL bài viết đã có từ file JSONL."""
    urls = set()
    if os.path.exists(file_path):
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                try:
                    urls.add(json.loads(line)['url'])
                except (json.JSONDecodeError, KeyError):
                    continue
    return urls

def convert_to_xlsx(json_path, xlsx_path):
    """Chuyển file JSON (dạng list các object) hoặc JSONL sang XLSX."""
    try:
        # Tự động phát hiện định dạng file
        if json_path.endswith('.jsonl'):
            df = pd.read_json(json_path, lines=True)
        else:
            df = pd.read_json(json_path) # Đọc file JSON chuẩn
            
        column_order = ["category", "sub_category", "url", "title", "description", "content", "date", "words"]
        df = df[[col for col in column_order if col in df.columns]]
        df.to_excel(xlsx_path, index=False, engine='openpyxl')
        print(f"-> Đã xuất thành công file Excel tại {xlsx_path}")
    except (FileNotFoundError, ValueError) as e:
        print(f"-> Không có dữ liệu hoặc lỗi khi chuyển sang Excel: {e}")

def get_url_key(item):
    match = re.search(r'-(\d+)\.html', item['url'])
    return int(match.group(1)) if match else 0

def heapify(arr, n, i, key_func):
    largest = i
    l = 2 * i + 1
    r = 2 * i + 2
    if l < n and key_func(arr[l]) > key_func(arr[largest]): largest = l
    if r < n and key_func(arr[r]) > key_func(arr[largest]): largest = r
    if largest != i:
        arr[i], arr[largest] = arr[largest], arr[i]
        heapify(arr, n, largest, key_func)

def heapSort(arr, key_func):
    n = len(arr)
    for i in range(n // 2 - 1, -1, -1):
        heapify(arr, n, i, key_func)
    for i in range(n - 1, 0, -1):
        arr[i], arr[0] = arr[0], arr[i]
        heapify(arr, i, 0, key_func)
    return arr

In [3]:
type_dict = load_json("Resource/categories.json")

my_config = {
    "BASE_URL": "https://vnexpress.net",

    "MIN_YEAR": 2020,
    "MIN_WORDS": 200,
    "MAX_WORDS": 1000,
    
    "TARGET_ARTICLES_PER_SUBTYPE": 5, # Số data cần cho mỗi subcategory
    "MAX_CONCURRENT_WORKERS": 6,
    "VALIDATION_ARTICLES_COUNT": 5,
    "PROGRESS_TIMEOUT": 10,
    "ARTICLE_TIMEOUT": 10,
    "MAX_CONSECUTIVE_FAILURES": 3,
    "URL_MAX_SUBCATEGORY_FAILURES": 3, 
    "ARTICLE_TIMEOUT": 5,
    
    "TYPE_DICT": type_dict
}

resource_dir = "Resource"
database_dir = "Database"
pageName = "VNExpress"

CATE_FILE = f"{resource_dir}/{pageName}_CATE.json"
DICT_FILE = f"{resource_dir}/{pageName}_DICT.json"

URLS_PATH = f"{resource_dir}/{pageName}_URLS"
JSON_PATH = f"{database_dir}/JSON/{pageName}"
XLSX_PATH = f"{database_dir}/XLSX/{pageName}"

os.makedirs(resource_dir, exist_ok=True)
os.makedirs(os.path.join(database_dir, "JSON"), exist_ok=True)
os.makedirs(os.path.join(database_dir, "XLSX"), exist_ok=True)

validated_dict = load_json(DICT_FILE)
categories_to_crawl = [
    'thoi-su', 
    'the-gioi', 
    'kinh-doanh', 
    'bat-dong-san', 
    'khoa-hoc', 
    'giai-tri', 
    'the-thao', 
    'phap-luat', 
    'giao-duc', 
    'suc-khoe', 
    'doi-song', 
    'oto-xe-may'
]

In [4]:
# --- Giai đoạn 1: Lấy danh sách chuyên mục hợp lệ ---
def getCategories():
    validator = CategoryValidator(config=my_config)
    valid_categories = validator.run()
    replace_json(valid_categories, DICT_FILE)

In [5]:
# --- Giai đoạn 2: Thu thập URL ---
def getDict():
    for category_name in categories_to_crawl:

        URLS_FILE = f"{URLS_PATH}_{category_name}.json"
        JSON_FILE = f"{JSON_PATH}_{category_name}.jsonl"

        print("\n" + "="*50)
        print(f"BẮT ĐẦU LẤY URL CHO CATEGORY: {category_name.upper()}")
        print("="*50)

        # 1. Lấy danh sách TẤT CẢ các URL đã tồn tại từ cả 2 nguồn
        urls_in_url_file = get_urls_from_url_file(URLS_FILE)
        urls_in_jsonl_file = get_existing_article_urls(JSON_FILE)
        all_existing_urls = urls_in_url_file.union(urls_in_jsonl_file)
        
        print(f"Đã tìm thấy {len(all_existing_urls)} URL đã tồn tại cho category này.")

        # 2. Chạy collector để thu thập tất cả URL có thể có
        url_collector = UrlCollector(config=my_config)
        all_possible_urls = url_collector.run(
            valid_subcategories=validated_dict,
            categories_to_process=[category_name]
        )

        # 3. Lọc ra chỉ những URL thực sự mới
        new_urls_info = [
            info for info in all_possible_urls 
            if info['url'] not in all_existing_urls
        ]
        
        print(f"Thu thập được {len(new_urls_info)} URL mới.")

        # 4. Ghi lại file URLS với dữ liệu cũ + mới
        if new_urls_info:
            existing_urls_info = load_json(URLS_FILE)
            combined_urls_info = existing_urls_info + new_urls_info
            replace_json(combined_urls_info, URLS_FILE)
    
# === END ===

In [6]:
# --- Giai đoạn 3: Crawl nội dung bài viết ---
def runCrawl():
    for category_name in categories_to_crawl:

        URLS_FILE = f"{URLS_PATH}_{category_name}.json"
        JSON_FILE = f"{JSON_PATH}_{category_name}.jsonl"

        print("\n" + "="*50)
        print(f"BẮT ĐẦU CRAWL CHO CATEGORY: {category_name.upper()}")
        print("="*50)

        urls_for_category = load_json(URLS_FILE)
        if not urls_for_category:
            print("Không có URL nào trong file để crawl. Bỏ qua.")
            continue

        # 1. Xác định các URL đã được xử lý từ trước
        existing_urls_in_jsonl = get_existing_article_urls(JSON_FILE)
        print(f"Tìm thấy {len(existing_urls_in_jsonl)} bài viết đã tồn tại trong file {JSON_FILE}.")
        
        # 2. Chạy crawler
        article_crawler = ArticleCrawler(config=my_config)
        new_articles, successfully_crawled_urls = article_crawler.run(
            urls_to_crawl=urls_for_category,
            category=category_name,
            existing_article_urls=existing_urls_in_jsonl
        )
        
        # 3. Lưu dữ liệu mới nếu có
        if new_articles:
            save_jsonl(new_articles, JSON_FILE)
            print(f"Đã lưu {len(new_articles)} bài báo mới vào {JSON_FILE}.")

        # 4. Xóa các URL đã được xử lý khỏi file URLS_FILE
        processed_urls = existing_urls_in_jsonl.union(set(successfully_crawled_urls))
        if processed_urls:
            # Lọc ra danh sách các URL còn lại
            remaining_urls = [
                info for info in urls_for_category 
                if info['url'] not in processed_urls
            ]
            
            # Ghi đè lại file URLS_FILE
            replace_json(remaining_urls, URLS_FILE)
            print(f"Đã xóa {len(processed_urls)} URL đã xử lý. Còn lại {len(remaining_urls)} URL trong hàng đợi.")

# === END ===

In [7]:
# --- Giai đoạn 4: Sort bài viết ---
def sortCrawl():
    for category_name in categories_to_crawl:

        URLS_FILE = f"{URLS_PATH}_{category_name}.json"
        JSON_FILE = f"{JSON_PATH}_{category_name}.jsonl"

        print("\n" + "="*50)
        print(f"BẮT ĐẦU SORT CHO CATEGORY: {category_name.upper()}")
        print("="*50)

        # --- Sắp xếp URLS_FILE ---
        urls_data = load_json(URLS_FILE)
        if urls_data:
            sorted_urls = heapSort(urls_data, key_func=get_url_key)
            replace_json(sorted_urls, URLS_FILE)

        # --- Sắp xếp JSON_FILE ---
        all_articles = load_jsonl(JSON_FILE)
        if all_articles:
            sorter = ArticleSorter(categories_file_path=CATE_FILE)
            sorted_articles = sorter.sort_and_deduplicate(all_articles)
            replace_jsonl(sorted_articles, JSON_FILE)
        
        print(f"-> Đã hoàn tất sắp xếp cho category: {category_name}")

# === END ===

In [8]:
# --- Giai đoạn 5: Tổng hợp và Hoàn thiện Dữ liệu (Logic cuối cùng, tự động) ---
def finalizeData():
    # Định nghĩa đường dẫn cho các file master
    MASTER_JSONL_FILE = f"{JSON_PATH}.jsonl"
    MASTER_JSON_FILE = f"{JSON_PATH}.json" 
    MASTER_XLSX_FILE = f"{XLSX_PATH}.xlsx"

    categories_dict = load_json(CATE_FILE)
    if not categories_dict:
        return
    
    all_categories = list(categories_dict.keys())

    # Xóa các file tổng hợp cũ để bắt đầu lại từ đầu
    for f in [MASTER_JSONL_FILE, MASTER_JSON_FILE]:
        if os.path.exists(f):
            os.remove(f)

    # 1. Gộp dữ liệu từ các file JSONL của tất cả category tìm được
    print("\n--- Bước 1: Gộp dữ liệu từ các file category ---")
    for category_name in all_categories: # Sử dụng danh sách vừa đọc được
        JSON_FILE = f"{JSON_PATH}_{category_name}.jsonl"
        # Chỉ xử lý nếu file của category đó tồn tại
        if os.path.exists(JSON_FILE):
            articles_data = load_jsonl(JSON_FILE)
            if articles_data:
                save_jsonl(articles_data, MASTER_JSONL_FILE)

    # 2. Sắp xếp và xóa trùng lặp file tổng hợp
    print("\n--- Bước 2: Sắp xếp và dọn dẹp file tổng hợp ---")
    all_merged_articles = load_jsonl(MASTER_JSONL_FILE)
    if all_merged_articles:
        sorter = ArticleSorter(categories_file_path=CATE_FILE)
        final_sorted_articles = sorter.sort_and_deduplicate(all_merged_articles)

        # 3. Ghi kết quả ra cả 2 định dạng file
        print("\n--- Bước 3: Ghi file tổng hợp ở cả 2 định dạng (.jsonl và .json) ---")
        replace_jsonl(final_sorted_articles, MASTER_JSONL_FILE)
        replace_json(final_sorted_articles, MASTER_JSON_FILE)

        # 4. Xuất file Excel cuối cùng
        print("\n--- Bước 4: Xuất file Excel tổng hợp ---")
        convert_to_xlsx(MASTER_JSON_FILE, MASTER_XLSX_FILE)

# === END ===

In [9]:
# getCategories()

In [10]:
# getDict()

In [11]:
# runCrawl()

In [12]:
# sortCrawl()

In [13]:
# finalizeData()

TRAIN and TEST

In [14]:
import pandas as pd
from transformers import pipeline
from Libraries import Trainer

training_config = {
    # --- Đường dẫn và tên model ---
    "DATA_JSONL_FILE": f"{JSON_PATH}.jsonl",
    "MODEL_CHECKPOINT": "vinai/bartpho-syllable",
    "OUTPUT_MODEL_DIR": "Models/bartpho-summarizer",
    
    # --- Hyperparameters ---
    "MAX_INPUT_LENGTH": 1024,
    "MAX_TARGET_LENGTH": 256,
    "BATCH_SIZE": 4,
    "NUM_TRAIN_EPOCHS": 3,
    "LEARNING_RATE": 3e-5,
    "WEIGHT_DECAY": 0.01,
}

In [15]:
# === TRAIN ===
def modelTrain():
    summarizer_trainer = Trainer.SummarizationTrainer(config=training_config)
    summarizer_trainer.run()

In [21]:
# === TEST ===
def modelTest():
    fine_tuned_model_path = training_config["OUTPUT_MODEL_DIR"] 
    summarizer_pipeline = pipeline("summarization", model=fine_tuned_model_path)

    # Lấy một bài báo từ dữ liệu của bạn để tóm tắt thử
    df = pd.read_json(training_config["DATA_JSONL_FILE"], lines=True)
    sample_text = df.iloc[400]["content"] # Số 50

    print("--- VĂN BẢN GỐC ---")
    print(sample_text)
    print("\n" + "="*50 + "\n")

    print("--- BẢN TÓM TẮT TỪ MODEL ---")
    summary = summarizer_pipeline(sample_text, max_length=256, min_length=50, do_sample=False)
    print(summary[0]['summary_text'])

In [17]:
# modelTrain()

In [22]:
modelTest()

--- VĂN BẢN GỐC ---
Thông tin trên được ông Vũ Bá Phú, Cục trưởng Cục Xúc tiến thương mại cùng các doanh nghiệp và hiệp hội ngành nghề, thông tin trong tọa đàm: "Thực trạng và thách thức trong xuất khẩu hàng sang Mỹ" sáng 8/8 tại TP HCM. Ông Phú cho rằng Mỹ từ lâu được xem như "mỏ vàng" của thương mại toàn cầu, với sức mua trải rộng từ áo quần giá 1 USD đến vài trăm USD mỗi chiếc, hay trái cây từ loại phổ thông đến cao cấp. Theo ông, dù thuế nhập khẩu vào Mỹ tăng lên 20% hay 40%, hàng Việt vẫn phải tìm đường vào, do đó, làm thế nào để hàng xuất đi có lợi nhất. Với quy định mới, hàng xuất xứ Việt Nam sẽ chịu thuế 20%, trong khi hàng "mượn xuất xứ" phải gánh 40%. Sáu tháng đầu năm, xuất khẩu sang Mỹ tăng mạnh, kéo theo lượng tồn kho lớn, kịch bản từng xảy ra sau Covid-19. Số liệu Cục Thống kê cho thấy 7 tháng qua, xuất siêu sang Mỹ đạt gần 75 tỷ USD, tăng 28,6% so với cùng kỳ năm ngoái. Tuy nhiên, ông Phú dự báo từ nửa cuối 2025, thậm chí sang nửa đầu 2026, nhiều ngành khó duy trì tốc độ